# WRDS Market Data Bars ( WRDS TAQ)

WRDS TAQ has millisecond-level best bid and offer data.  However, downloading the full data series is not practical due to the hundreds of millions of rows per day.  Here, we use SQL to generate more manageable "bars" on the server side.

In [1]:
import pandas as pd
import wrds

In [2]:
import datetime
import pytz

In [3]:
from functools import lru_cache

In [4]:
db = wrds.Connection(
    wrds_username=BrianBoonstraPrivateKeys['wrds']['username'],
)

Loading library list...
Done


In [5]:
', '.join([t for t in db.list_tables(library='taqm_2024') if '2024' not in t or '20240124' in t])

'complete_nbbo_20240124, cqm_20240124, ctm_20240124, luld_cqm_20240124, luld_ctm_20240124, mastm_20240124, nbbom_20240124, wct_20240124'

`complete_nbbo` is often sufficient. `cqm` is all quotes, but not enough to build the book.  `nbbom` is by exchange. `luld` is limit up limit down.  

## National Best Bid And Offer

We use a PostgreSQL window function and partitioning to identify which rows are relevant to bars. That way, our table join gets only a few rows.

For efficiency, we do not use the painfully-more-expensive GROUP BY, and hence do not compute update counts, minima, or maxima.

After the query we clean up the date and time data a little

**Note:** Bars in which there were no quotes will be missing from the output

In [22]:
@lru_cache
def taq_nbbo_bars_on_date(tickers: list[str] | str, date:datetime.date, bar_minutes=30):
    assert bar_minutes==60 or (bar_minutes<=30 and 30%bar_minutes==0)
    date_str = date.strftime('%Y%m%d')
    year_str = date.strftime('%Y')
    tickers = (tickers,) if hasattr(tickers,'strip') else tuple(tickers)  # Allow single ticker as argument

    sql = f"""
            WITH windowable_nbbo AS (
                SELECT
                    sym_root AS ticker
                    , date
                    , time_m
                    , time_m_nano
                    , sym_root
                    , qu_cond
                    , best_bid
                    , best_bidsizeshares
                    , best_ask
                    , best_asksizeshares
                    , EXTRACT(HOUR FROM time_m) AS hour_of_day
                    , {bar_minutes} * DIV(EXTRACT(MINUTE FROM time_m),{bar_minutes}) AS minute_of_hour
                    , ROW_NUMBER() OVER (PARTITION BY sym_root, EXTRACT(HOUR FROM time_m), DIV(EXTRACT(MINUTE FROM time_m),{bar_minutes}) ORDER BY time_m DESC) AS rownum
                FROM taqm_{year_str}.complete_nbbo_{date_str} 
                WHERE 1=1
                  AND sym_root IN {tickers}
                  AND sym_suffix IS NULL
                  AND time_m > '09:30:00' AND time_m < '16:00:00'
            )
            SELECT DISTINCT ON (ticker, date, hour_of_day, minute_of_hour)
                ticker
                , date
                , date + (hour_of_day || ':' || minute_of_hour)::interval + ( '00:{bar_minutes}' )::interval AS window_time
                , best_bid
                , best_bidsizeshares
                , best_ask
                , best_asksizeshares
                , time_m AS time_of_last_quote
                , time_m_nano AS time_of_last_quote_ns
            FROM windowable_nbbo
            WHERE windowable_nbbo.rownum = 1
            """
    bars = db.raw_sql(sql)
    
    def _make_timestamp(r):
        t = datetime.datetime.combine(r.date, r.time_of_last_quote)#, tzinfo=pytz.timezone('America/New_York'))
        pdt = pd.to_datetime(t).tz_localize(pytz.timezone('America/New_York')) + pd.Timedelta(r.time_of_last_quote_ns, unit="ns")
        return pdt
    bars['time_of_last_quote'] = bars.apply(_make_timestamp, axis=1)
    del bars['time_of_last_quote_ns']
    bars['window_time'] = pd.to_datetime(bars['window_time']).dt.tz_localize(pytz.timezone('America/New_York'))
    return bars

In [23]:
# Execution time about 10 seconds
b20240229 = taq_nbbo_bars_on_date(('SPY', 'PBPB', 'HLIT'), date=datetime.date(2024,2,29))
b20240229.set_index(['ticker', 'date', 'window_time'])

minute_of_hour  best_bid  \
ticker date       window_time                                           
HLIT   2024-02-29 2024-02-29 10:00:00-05:00            30.0     13.26   
                  2024-02-29 10:30:00-05:00             0.0     13.11   
                  2024-02-29 11:00:00-05:00            30.0     13.13   
                  2024-02-29 11:30:00-05:00             0.0     13.04   
                  2024-02-29 12:00:00-05:00            30.0     13.01   
                  2024-02-29 12:30:00-05:00             0.0     13.03   
                  2024-02-29 13:00:00-05:00            30.0     13.05   
                  2024-02-29 13:30:00-05:00             0.0     13.09   
                  2024-02-29 14:00:00-05:00            30.0     13.09   
                  2024-02-29 14:30:00-05:00             0.0     13.11   
                  2024-02-29 15:00:00-05:00            30.0     13.11   
                  2024-02-29 15:30:00-05:00             0.0     13.14   
                  2024-02-29 16:00:00-05:00            30.0     13.13   
PBPB   2024-02-29 2024-02-29 10:00:00-05:00            30.0     13.74   
                  2024-02-29 10:30:00-05:00             0.0     13.68   
                  2024-02-29 11:00:00-05:00            30.0     13.75   
                  2024-02-29 11:30:00-05:00             0.0     13.73   
                  2024-02-29 12:00:00-05:00            30.0     13.81   
                  2024-02-29 12:30:00-05:00             0.0     13.83   
                  2024-02-29 13:00:00-05:00            30.0     13.84   
                  2024-02-29 13:30:00-05:00             0.0     13.84   
                  2024-02-29 14:00:00-05:00            30.0     13.85   
                  2024-02-29 14:30:00-05:00             0.0     13.83   
                  2024-02-29 15:00:00-05:00            30.0     13.86   
                  2024-02-29 15:30:00-05:00             0.0     13.86   
                  2024-02-29 16:00:00-05:00            30.0     13.88   
SPY    2024-02-29 2024-02-29 10:00:00-05:00            30.0    508.49   
                  2024-02-29 10:30:00-05:00             0.0    507.69   
                  2024-02-29 11:00:00-05:00            30.0    507.44   
                  2024-02-29 11:30:00-05:00             0.0    507.18   
                  2024-02-29 12:00:00-05:00            30.0    506.65   
                  2024-02-29 12:30:00-05:00             0.0    506.58   
                  2024-02-29 13:00:00-05:00            30.0    507.36   
                  2024-02-29 13:30:00-05:00             0.0    507.76   
                  2024-02-29 14:00:00-05:00            30.0    507.80   
                  2024-02-29 14:30:00-05:00             0.0    507.71   
                  2024-02-29 15:00:00-05:00            30.0    507.74   
                  2024-02-29 15:30:00-05:00             0.0    508.13   
                  2024-02-29 16:00:00-05:00            30.0    507.99   

                                             best_bidsizeshares  best_ask  \
ticker date       window_time                                               
HLIT   2024-02-29 2024-02-29 10:00:00-05:00                 600     13.27   
                  2024-02-29 10:30:00-05:00                 400     13.12   
                  2024-02-29 11:00:00-05:00                 500     13.14   
                  2024-02-29 11:30:00-05:00                 400     13.05   
                  2024-02-29 12:00:00-05:00                 700     13.02   
                  2024-02-29 12:30:00-05:00                 700     13.04   
                  2024-02-29 13:00:00-05:00                1100     13.06   
                  2024-02-29 13:30:00-05:00                 400     13.10   
                  2024-02-29 14:00:00-05:00                1000     13.10   
                  2024-02-29 14:30:00-05:00                 600     13.12   
                  2024-02-29 15:00:00-05:00                 500     13.12   
                  2024-02-29 15:30:00-05:00       